In [ ]:
%matplotlib inline
from livelossplot import PlotLossesKeras
from keras.callbacks import EarlyStopping
from keras.callbacks import LearningRateScheduler
from src import camvid
from src import tiramisu

In [ ]:
!mkdir -p models

In [ ]:
# the size of the images in the dataset
target_size = (720, 960)
# the size to crop images to for training
coarse_crop = (224, 224)
# the size to crop images to for training
# fine_crop = (704, 960)
fine_crop = (320, 320)

# Course Tuning - Train on Random Crops

In [ ]:
generators, transformer = camvid.data_generators(target_size=target_size, crop_size=coarse_crop)

In [ ]:
# get the next X, y training tuple
X, y = next(generators['training'])
# transform the onehot vector to an image
y = transformer.unmap(y)
# plot the images
camvid.plot(X[0], y[0])

In [ ]:
# get the size of the one-hot vectors from the transformer
num_labels = len(transformer)
# build the model for the image shape and number of labels
model = tiramisu.build_tiramisu((*coarse_crop, 3), num_labels,
    label_names=transformer.labels,
    learning_rate=1e-3,
)

In [ ]:
# fit the model with the data
history = model.fit_generator(generators['training'],
    epochs=250,
    steps_per_epoch=491,
    validation_data=generators['validation'],
    validation_steps=210,
    callbacks=[
        LearningRateScheduler(lambda _, lr: 0.995 * lr),
        EarlyStopping(monitor='val_acc', patience=100),
        PlotLossesKeras(),
    ],
    verbose=0
)

In [ ]:
X, y = next(generators['validation'])
p = model.predict(X)
camvid.plot(X[0], transformer.unmap(y)[0], transformer.unmap(p)[0])

In [ ]:
asdf

## Save Model

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open('models/tiramisu103-coarse.json', 'w') as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights('models/tiramisu103-coarse.h5')

# Fine Tuning - Train on Whole Images

In [ ]:
generators, transformer = camvid.data_generators(target_size=target_size, crop_size=fine_crop)

In [ ]:
# get the next X, y training tuple
X, y = next(generators['training'])
# transform the onehot vector to an image
y = transformer.unmap(y)
# plot the images
camvid.plot(X[0], y[0])

In [ ]:
# # get the size of the one-hot vectors from the transformer
num_labels = len(transformer)
# build the model for the image shape and number of labels
tune_model = tiramisu.build_tiramisu((*fine_crop, 3), num_labels,
    label_names=transformer.labels,
    learning_rate=1e-4,
)
# load the weights from the coarsely trained model
tune_model.set_weights(model.get_weights())

In [ ]:
# fit the model with the data
history = tune_model.fit_generator(generators['training'],
    epochs=250,
    steps_per_epoch=491,
    validation_data=generators['validation'],
    validation_steps=210,
    callbacks=[
        LearningRateScheduler(lambda _, lr: 0.995 * lr),
        EarlyStopping(monitor='val_acc', patience=50),
        PlotLossesKeras(),
    ],
    verbose=0
)

In [ ]:
X, y = next(generators['validation'])
p = tune_model.predict(X)
camvid.plot(X[0], transformer.unmap(y)[0], transformer.unmap(p)[0])

## Save Model

In [ ]:
# serialize model to JSON
model_json = tune_model.to_json()
with open('models/tiramisu103-fine.json', 'w') as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
tune_model.save_weights('models/tiramisu103-fine.h5')